<a href="https://colab.research.google.com/github/86HenriqueSilva/mega_sena/blob/main/03_Mega_Sena_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm
import pandas as pd
import gdown

# Função para pós-processamento das previsões
def postprocess_predictions(predictions):
    # Lista para armazenar as previsões únicas para cada bola
    unique_predictions = []

    # Loop sobre cada bola
    for i in range(6):
        # Encontrar uma previsão única para a bola atual
        unique_value = np.random.choice(np.setdiff1d(range(1, 61), unique_predictions))
        unique_predictions.append(unique_value)

    return unique_predictions

# URL do arquivo de dados
data_url = 'https://drive.google.com/uc?id=1t28TCezmfyBc5X_3-JwqN-FxkV63M-t5'

# Baixar o arquivo de dados e carregá-lo como DataFrame
data = pd.read_csv(data_url)

# Remover linhas com valores não numéricos
data = data.dropna()

# Remover a coluna de data
data = data.drop(columns=['Data'])

# Dividir os dados em features (X) e targets (y)
X = data.drop(columns=['Bola 1', 'Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6'])
y = data[['Bola 1', 'Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6']]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1024)

# Padronizar as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construir o modelo da rede neural
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
epochs = 10000
batch_size = 128
with tqdm(total=epochs, desc="Treinamento da Rede Neural") as pbar:
    for epoch in range(epochs):
        pbar.update(1)
        model.fit(X_train_scaled, y_train, batch_size=batch_size, verbose=0)
        pbar.set_postfix({'Epoch': epoch + 1})

# Avaliar o modelo
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)

# Exibir as previsões pós-processadas para o próximo concurso
print("Previsões pós-processadas para o próximo sorteio:")
y_pred_int_postprocessed = postprocess_predictions(y_pred.flatten())
for i, pred in enumerate(y_pred_int_postprocessed):
    print(f"Bola {i+1}: {pred}")

# Exibir o erro médio absoluto no conjunto de teste
print("Erro médio absoluto no conjunto de teste:", mae)


Treinamento da Rede Neural:   0%|          | 0/10000 [00:00<?, ?it/s]

35/35 [==============================] - 0s 2ms/step
Previsões pós-processadas para o próximo sorteio:
Bola 1: 7
Bola 2: 29
Bola 3: 19
Bola 4: 8
Bola 5: 42
Bola 6: 18
Erro médio absoluto no conjunto de teste: 14.939218766243618
